<a href="https://colab.research.google.com/github/gmb408/OSSP-KBO-prediction/blob/version-0.1/predict_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#이 부분은 처음 한번만 실행하면 됌.
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 7.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubun

In [ ]:
# 데이터 크롤링 모듈
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import re


In [ ]:
# 데이터 분석 모듈
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime

In [13]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [14]:
for i in range(4):
    
    # 1982년 부터 2018년 까지 statiz에 기록된 선수들 필터링 (총 8558명)
    url = 'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2022&ye=2022&sn=100&pa={}'.format(i*100)
    
    driver.get(url)
    driver.implicitly_wait(5)
    html = driver.find_element(By.XPATH,'//*[@id="mytable"]/tbody').get_attribute("innerHTML") #기록 table을 str형태로 저장
    soup = BeautifulSoup(html, 'html.parser') #str 객체를 BeautifulSoup 객체로 변경
    
    temp = [i.text.strip() for i in soup.findAll("tr")] #tr 태그에서, text만 저장하기
    temp = pd.Series(temp) #list 객체에서 series 객체로 변경
    
    #'순'이나 'W'로 시작하는 row 제거
    # 즉, 선수별 기록만 남기고, index를 reset 해주기
    temp = temp[~temp.str.match("[순W]")].reset_index(drop=True) 
    
    temp = temp.apply(lambda x: pd.Series(x.split(' '))) #띄어쓰기 기준으로 나눠서 dataframe으로 변경
    
    #선수 팀 정보 이후 첫번째 기록과는 space 하나로 구분, 그 이후로는 space 두개로 구분이 되어 있음 
    #그래서 space 하나로 구분을 시키면, 빈 column들이 존재 하는데, 해당 column들 제거 
    temp = temp.replace('', np.nan).dropna(axis=1) 
    
    #WAR 정보가 들어간 column이 2개 있다. (index가 1인 column과, 제일 마지막 column)
    #그 중에서 index가 1인 columm 제거 
    temp = temp.drop(1, axis=1)
    
    #선수 이름 앞의 숫자 제거
    temp[0] = temp[0].str.replace("^\d+", '')

    # 선수들의 생일 정보가 담긴 tag들 가지고 오기
    birth = [i.find("a") for i in soup.findAll('tr') if 'birth' in i.find('a').attrs['href']]
    
    # tag내에서, 생일 날짜만 추출하기 
    p = re.compile("\d{4}\-\d{2}\-\d{2}")
    birth = [p.findall(i.attrs['href'])[0] for i in birth]
    
    # 생일 column 추가
    temp['생일'] = birth
    
    # page별 완성된 dataframe을 계속해서 result에 추가 시켜주기 
    if i == 0:
        result = temp
    else:
        result = result.append(temp)
        result = result.reset_index(drop=True)
        
    print(i, "완료")
        
#column 명 정보 저장        
columns = ['선수'] + [i.text for i in soup.findAll("tr")[0].findAll("th")][4:-3] + ['타율', '출루', '장타', 'OPS', 'wOBA', 'wRC+', 'WAR+', 'WPA','생일']

#column 명 추가
result.columns = columns

#webdriver 종료
driver.close()

print("최종 완료")

<ipython-input-14-c29520274c69>:29: FutureWarning: The default value of regex will change from True to False in a future version.
  temp[0] = temp[0].str.replace("^\d+", '')


0 완료
1 완료
2 완료
3 완료
최종 완료


In [15]:
#결과값 저장해두기
result.to_csv("/content/pridiction.csv")
result

,선수,G,타석,타수,득점,안타,2타,3타,홈런,루타,...,희비,타율,출루,장타,OPS,wOBA,wRC+,WAR+,WPA,생일
0,이정후22키CF,142,627,553,85,193,36,10,23,318,...,3,.349,.421,.575,.996,.441,182.5,9.23,6.72,1998-08-20
1,피렐라22삼LF,141,630,561,102,192,33,4,28,317,...,2,.342,.411,.565,.976,.434,169.3,7.40,4.20,1989-11-21
2,나성범22KRF,144,649,563,92,180,39,2,21,286,...,5,.320,.402,.508,.910,.411,157.4,6.50,3.74,1989-10-03
3,오지환22LSS,142,569,494,75,133,16,4,25,232,...,3,.269,.357,.470,.827,.372,138.6,5.77,2.56,1990-03-12
4,최정22S3B,121,505,414,80,110,21,0,26,209,...,6,.266,.386,.505,.891,.400,145.4,5.15,3.72,1987-02-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,조세진22롯RF,39,88,86,6,16,3,0,0,19,...,0,.186,.195,.221,.416,.192,5.7,-0.76,-0.64,2003-11-21
353,유로결22한LF,30,68,60,5,8,0,0,0,8,...,0,.133,.200,.133,.333,.164,-11.4,-0.82,-1.05,2000-05-30
354,정보근22롯C,95,226,199,8,38,2,0,1,43,...,1,.191,.250,.216,.466,.218,24.1,-0.91,-1.78,1999-08-31
355,박경수22K2B,100,194,166,13,20,3,0,3,32,...,1,.120,.234,.193,.427,.213,21.9,-1.01,-2.16,1984-03-31
